**Как пользоваться: в единственной ячейке вводишь данные по тикерам и нажимаешь в верхней части Среда выполнения -> Выполнить все (Ctrl + F9).**

График котировок строится по High цене, в качестве цены покупки можно выбрать High, Low или среднюю между ними цену (поле buy_price в ячейке параметров).

In [ ]:
#@title
%%capture
!pip install pandas_market_calendars

In [ ]:
#@title
import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas_market_calendars as mcal

In [ ]:
#@title Введите тикеры акций, дату покупки, ожидаемую прибыль в процентах, выберите цену покупки (High, Low, Median)
#@markdown
 
tickers = 'TER, RIO, SHAK, AMCR, ZUMZ, IRTC, SFIX, SMPL, VRSN, FTNT, RPD, LYV'  #@param {type: "string"}
buy_date = '2021-03-02'  #@param {type: "date"}
desired_profit = 3.05  #@param {type: "number"}
buy_price = "Median"  #@param ['High', 'Low', 'Median']
#@markdown ---

In [ ]:
#@title
def parse_tickers():
  # разбираем строчку тикеров
  tickers_list = "".join(tickers.split())
  tickers_list = tickers_list.split(',')
  return tickers_list

In [ ]:
#@title
def input_data_process(ticker):
  buy_date_list = list(map(int, buy_date.split('-')))
  pd_start_date = pd.Timestamp(buy_date)

  # берём временной интервал от введённой даты покупки до вчерашнего дня,
  # т.к. за сегодня ещё нет окончательных данных
  start = dt.datetime(year=buy_date_list[0], month=buy_date_list[1], day=buy_date_list[2])
  end = dt.datetime.now() - dt.timedelta(days = 1)

  #определение цены покупки
  price_on_buy_date = web.DataReader([ticker], 'yahoo', start, start)
  if buy_price == 'High':
    buy_price_usd = price_on_buy_date['High']
  elif buy_price == 'Low':
    buy_price_usd = price_on_buy_date['Low']
  elif buy_price == 'Median':
    buy_price_usd = (price_on_buy_date['High'] + price_on_buy_date['Low']) / 2  

  # определяем дни торгов в рассматриваемом интервале дат
  nyse = mcal.get_calendar('NYSE')
  open_markets_days = nyse.schedule(start_date=pd.Timestamp(buy_date), end_date=pd.Timestamp(end))    

  return pd_start_date, start, end, buy_price_usd, open_markets_days

In [ ]:
#@title
def get_quotes(ticker, start, end, buy_price_usd):
  stock_prices = web.DataReader([ticker], 'yahoo', start, end)
  stock_high = pd.DataFrame(stock_prices)['High']

  # расчёт ожидаемой цены с учётом заданной доходности  
  desired_price = buy_price_usd[ticker].item() * (1 + float(desired_profit) / 100)

  days_to_get_profit = None
  for index, row in stock_high.iterrows(): 
      if (row.values >= desired_price):
          days_to_get_profit = index.date() - pd_start_date.date()
          break

  return desired_price, stock_high, days_to_get_profit

In [ ]:
#@title
def print_chart(ticker, buy_price_usd, desired_price, stock_high, days_to_get_profit, open_markets_days):
  fig, ax = plt.subplots(figsize=(16,9))

  print(ticker)
  buy_price_usd_format = format(buy_price_usd[ticker].item(), '.2f')
  print('цена покупки :', buy_price_usd_format)
  desired_price_format = format(desired_price, '.2f')
  print('цена с учётом ожидаемой доходности', desired_profit, 'процентов :', desired_price_format)
  if days_to_get_profit is None:
    print('ожидаемая доходность не достигнута')  
  else:
    print('достижение ожидаемой доходности заняло', days_to_get_profit.days, 'дней')

  tick = stock_high.loc[:, ticker]
  ax.plot(tick.index, tick, label=ticker)
  ax.plot(open_markets_days.axes[0], [desired_price] * open_markets_days.shape[0], label='Desired price, $')
  ax.set_xlabel('Date')
  ax.set_ylabel('Close price, $')
  ax.legend()
  ax.grid()  
  plt.pause(0.05)

In [ ]:
#@title
tickers_list = parse_tickers()
for ticker in tickers_list:
  pd_start_date, start, end, buy_price_usd, open_markets_days = input_data_process(ticker)
  desired_price, stock_high, days_to_get_profit = get_quotes(ticker, start, end, buy_price_usd)
  print_chart(ticker, buy_price_usd, desired_price, stock_high, days_to_get_profit, open_markets_days)